In [ ]:
!pip install diffusers transformers accelerate -q

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

# load pipeline
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# optimize for GPU memory
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

In [ ]:
# generate
prompt = "sloth driving race car"
video_frames = pipe(prompt, num_inference_steps=10, num_frames=200).frames

In [ ]:
# convent to video
video_path = export_to_video(video_frames[0])

In [ ]:
video_path

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = video_path

# Compressed video path
compressed_path = "/tmp/result_compressed.mp4"

# Delete the previous compressed file if it exists
if os.path.exists(compressed_path):
    os.remove(compressed_path)

# Compress the video
os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")
